In [0]:
#Transformations for Country_Quarter_Wise_Visitors.csv

from pyspark.sql.functions import col
input_path = '/mnt/bronze/Country_Quarter_Wise_Visitors.csv'

df = spark.read.format('csv').option("header", "true").load(input_path)

#1. Replace space with underscore in column names
#2. Changing column name from "2014 1st quarter (Jan-March)" to "2024_1" and like-wise

df = df.select([col(x).alias(x.strip().replace(' ','_')) for x in df.columns])

for column in df.columns:
     if "quarter" in column:
         df = df.withColumnRenamed(column, column[:6])

output_path = '/mnt/gold/Country_Quarter_Wise_Visitors'
df.write.format('delta').mode('overwrite').save(output_path)


In [0]:
#Transformations for Country_Wise_Airport.csv

input_path = '/mnt/bronze/Country_Wise_Airport.csv'

df = spark.read.format('csv').option("header", "true").load(input_path)
df = df.select([col(x).alias(x.strip().replace(' ','_')) for x in df.columns])

#Remove aiport from column name
for column in df.columns:
    if "Airport" in column:
        df = df.withColumnRenamed(column, column.replace('(Airport)','').strip().strip('_'))

output_path = '/mnt/gold/Country_Wise_Airport'
df.write.format('delta').mode('overwrite').save(output_path)

In [0]:
#Loading all other files to gold

transformed_files = ['Country_Quarter_Wise_Visitors.csv', 'Country_Wise_Airport.csv']

input_path = '/mnt/bronze/'

for file in dbutils.fs.ls(input_path):
    if file.name not in transformed_files:
        df = spark.read.format('csv').option("header", "true").load(file.path)
        df = df.select([col(x).alias(x.strip().replace(' ','_')) for x in df.columns])

        output_path = f'/mnt/gold/{file.name[:-4]}'
        df.write.format('delta').mode('overwrite').save(output_path)